In [2]:
import pandas as pd
import numpy as np
df = pd.read_csv("train-train.csv")

In [14]:
# -------------------------------
# data Exploration
# -------------------------------
'''
print(df.head())
print(df.describe())
print(df.info())
print(df["metaphorID"].unique())
'''
# -------------------------------
#check for missing values
# -------------------------------
print(df.isnull().sum())
print(df.duplicated().sum())

#get duplicate rows
duplicate_rows = df[df.duplicated(keep=False)]
print(duplicate_rows)

df_clean = df.drop_duplicates(keep='first')
print(df_clean.duplicated().sum())

metaphorID    0
label         0
text          0
dtype: int64
1
     metaphorID  label                                               text
50            0   True  Thanks , Tender . Yes , this whole recurrence ...
448           0   True  Thanks , Tender . Yes , this whole recurrence ...
0


In [15]:
# -------------------------------
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ------------- -------------------------- 4.2/12.8 MB 29.6 MB/s eta 0:00:01
     ---------------------------------------- 12.8/12.8 MB 44.4 MB/s  0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")
document = nlp.pipe(df['text'], disable=['ner', 'parser', 'textcat'])
df['spacy_text'] = list(document)
def tokenize_text(doc,join_tokens=False):
    tokens = [word.lemma_.lower() for word in doc if not word.is_stop and not word.is_punct and not word.like_num or (word.lemma_ == 'not')]
    if join_tokens:
        return " ".join(tokens)
    return tokens

df['clean_text'] = df['spacy_text'].apply(tokenize_text)


# Map metahor id with candidates
metaphor_candidates = {0: 'road', 1: 'candle', 2: 'light', 3: 'spice', 4: 'ride', 5: 'train', 6: 'boat'}
df['metaphor_candidate'] = df['metaphorID'].map(metaphor_candidates)

#df.head()

for sentence in enumerate(df['spacy_text'][:5]):
    print(sentence)

,metaphorID,label,text,spacy_text,clean_text,metaphor_candidate
0,0,True,"Hey , Karen !!!! I was told that on the day of...","(Hey, ,, Karen, !, !, !, !, I, was, told, that...","[hey, karen, tell, day, diagnosis, survivor, c...",road
1,2,False,"Hi Ladies ... my last chemo was Feb 17/09 , ra...","(Hi, Ladies, ..., my, last, chemo, was, Feb, 1...","[hi, ladies, chemo, feb, rad, finish, apr, ari...",light
2,2,False,I have just come form my consult with a lovely...,"(I, have, just, come, form, my, consult, with,...","[come, form, consult, lovely, ps, not, thin, l...",light
3,4,False,I also still question taking Tamox for stage 1...,"(I, also, still, question, taking, Tamox, for,...","[question, take, tamox, stage, grade, bc, lump...",ride
4,2,False,Just checking in to say hello ladies . I had a...,"(Just, checking, in, to, say, hello, ladies, ....","[check, hello, lady, long, busy, day, get, all...",light


In [53]:
df['text'][0]
candites = metaphor_candidates.values()
for sentence in df['spacy_text'][:2]:
    nlp_sentence = nlp(sentence.text)
    for sent in nlp_sentence.sents:
        for token in sent:
            if token.lemma_ in candites:
                print(sent)

You will do great , just a gliche in the road .
so now my question ... are any of you into running/jogging and if yes , are any of you having problems with feeling light headed ?


In [77]:
def get_candidate_sentence(row, candidate):
        nlp_sentence = nlp(row.text)
        for sent in nlp_sentence.sents:
            if candidate in [token.lemma_ for token in sent]:
                return sent.text

#df['candidate_sentence'] = df.apply(get_candidate_sentence)

df['candidate_sentence'] = df.apply(lambda row: get_candidate_sentence(row['spacy_text'], row['metaphor_candidate']), axis=1)

In [ ]:
import pandas as pd
import numpy as np
import spacy

nlp = spacy.load("en_core_web_sm")

# -------------------------------
# 1. Load dataset
# -------------------------------
def read_data(file_path):
    return pd.read_csv(file_path) # columns: Metaphor_ID, Label, Text

# -------------------------------
# Extract candidate sentence
# -------------------------------

def get_candidate_sentence(row, candidate):
    nlp_sentence = nlp(row)
    for sent in nlp_sentence.sents:
        for token in sent:
            str_token = str(token)
            if candidate in token.lemma_.lower() or candidate in  str_token.lower() :
                return sent.text

def tokenize(doc, join_tokens=False):
    print(doc)
    nlp_sentence = nlp(doc)
    tokens = [word.lemma_.lower() for word in nlp_sentence if not word.is_stop and not word.is_punct and not word.like_num or (word.lemma_ == 'not')]
    if join_tokens:
        return " ".join(tokens)
    return tokens


# Load data
initial_data = read_data("train-train.csv")
    
# Drop duplicates
data = initial_data.drop_duplicates(keep='first').reset_index(drop=True).copy()


# Map metahor id with candidates
metaphor_candidates = {0: 'road', 1: 'candle', 2: 'light', 3: 'spice', 4: 'ride', 5: 'train', 6: 'boat'}
data.loc[:, 'metaphor_candidate'] = data['metaphorID'].map(metaphor_candidates)

data.loc[:, 'candidate_sentence'] = data.apply(
    lambda row: get_candidate_sentence(row['text'], row.metaphor_candidate),
    axis=1
)

print(data['candidate_sentence'].isnull().sum())

#data.loc[:, 'tokenized_candidate_sentence'] = data['candidate_sentence'].apply(lambda row: tokenize(row))

#print(data['tokenized_candidate_sentence'].head())




road
['hey', ',', 'Karen', '!', '!', '!', '!']
road
['I', 'be', 'tell', 'that', 'on', 'the', 'day', 'of', 'diagnosis', 'you', 'be', 'a', 'survivor', 'and', 'you', 'certainly', 'will', 'be', '.']
road
['God', 'bless', 'you', 'and', 'my', 'prayer', 'be', 'with', 'you', 'as', 'you', 'go', 'through', 'treatment', '.']
road
['you', 'will', 'be', 'fine', '.']
road
['I', 'feel', 'pretty', 'good', 'and', 'I', 'have', '8', 'treatment', 'and', 'that', 'be', '3', 'and', '1/2', 'year', 'ago', '.']
road
['I', 'would', 'have', 'keep', 'work', 'but', 'I', 'teach', 'Kindergarten', 'at', 'the', 'time']
road
['and', 'I', 'be', 'afraid', 'of', 'all', 'the', 'germ', 'float', 'around', '.']
road
['however', ',', 'as', 'I', 'look', 'back', 'I', 'should', 'have', 'work', 'just', 'to', 'keep', 'busy', '.']
road
['you', 'will', 'do', 'great', ',', 'just', 'a', 'gliche', 'in', 'the', 'road', '.']
light
['hi', 'Ladies', '...', 'my', 'last', 'chemo', 'be', 'Feb', '17/09', ',', 'rad', 'finish', 'Apr', '29', 'and',

In [116]:
data[data['candidate_sentence'].isnull()].index
print(data['candidate_sentence'][40])
print(data['text'][40])
print(data['metaphor_candidate'][40])
print(data['metaphorID'][40])

def get_candidate_sentence(row, candidate):
    nlp_sentence = nlp(row)
    
   
    for sent in nlp_sentence.sents:
        for token in sent:
            str_token = str(token)
            if candidate in token.lemma_.lower() or candidate in  str_token.lower() :
                return sent.text
print(get_candidate_sentence(data['text'][40], data['metaphor_candidate'][40]))

None
Kim & Lisa I hope you both get the oncotype test to be sure chemo is helpful . My score came back at a 16 and 2 oncologist suggested me to just do tamoxifin . I 'm glad to say I have n't noticed bad side effects . Plus I want to share with you I had my surgery about 3ξ1/2 months ago and I 'm right back at my long bike rides and runs . I hope you ladies have a speedy recovery also : - -RRB-
ride
4
and I 'm right back at my long bike rides and runs .
